# 목표 : K-Nearest Neighbor 모델 구축
# 1. 데이터 전처리 
# 2. K-Nearest Neighbor 모델 구축 
## K-fold cross validation 활용
### - k 개수 선정 " 인접한 학습데이터를 몇 개까지 탐색할 것인가?"
### - Distance measure 선정 "데이터 간 거리를 어떻게 측정할 것인가?" 
# 3. 예측 결과 평가 및 weighted KNN 구축 

# ----------------------------------------------------------

# 1. 모듈 불러오기

In [13]:
from IPython.display import display, HTML
import warnings
warnings.filterwarnings(action="ignore")

#실습용 데이터 전처리
from sklearn.datasets import load_breast_cancer

#데이터 전처리 
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler

#기계학습 모델 및 평가 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, recall_score, precision_score
from tqdm import tqdm

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import KFold
from scipy.stats import spearmanr

#시각화
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
from matplotlib.colors import ListedColormap
matplotlib.rcParams["axes.unicode_minus"] = False # 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처

#한글폰트설정 
plt.rc("font", family="Malgun Gothic")



# 2. 데이터 불러오기 : Breast Cancer

In [14]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
print(breast_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

## 데이터 구조
### - 데이터: 위스콘신 대학교에서 제공한 유방암 진단결과
### - 관측치 개수: 569개
### - 변수 개수: 32개(ID:진단결과, 30개의 실측변수값)


## 설명변수(원인: 예측값을 설명할 수 있는 변수) 
### - id : 환자 식별 번호
### - radius : 반경(중심에서 외벽까지 거리들의 평균값)
### - texture : 질감(Gray-Scale 값들의 표준편차)
### - perimeter : 둘레
### - area : 면적
### - smoothness : 매끄러움(반경길이의 소극적 변화)
### - compactness : 조그만 정도(둘레^2/면적-1)
### - concavity : 오목함(윤곽의 오목한 정도)
### - points : 오목한 점의 수
### - symmetry : 대칭
### - dimension : 프렉탈 차원(해안선 근사-1)
### - mean : 3~12번 까지는 평균값을 의미
### - se : 13~22번까지는 표준오차를 의미 
### - worst : 23~32는 각 세포별 구분들에서 제일 큰 3개의 값을 평균낸 값


## 반응 변수(결과: 예측하고자 하는 값)
### - diagnosis : 양성여부(Maligant = 1 악성(유방암), Benign=0 양성(단순낭종))